# Constants

In [282]:
c = 137.036;  # speed of light in atomic units
eV = 0.0367493;  # 1 eV in atomic units
hbar = 6.582119569*10^(-16); # S.I. value in eV s 
nm = 100/5.29177210903; # 1nm in atomic units
q = -1.0; # charge of the electron in atomic units
ii = 0.0 + 1*im; # imaginary unit

## Dielectric function  of the nanoparticle

### Al-Drude

In [283]:
plasmafrequency = 13.14232*eV; # plasma frequency of Drude model
damping = 0.19713*eV; # damping of Drude model

epsilon(w) = 1 - (( plasmafrequency * plasmafrequency ) / ( w * ( w + ii * damping ))); # dielectric function of the nanoparticle

### Au Werner

#### Werner parameters

In [284]:
#omegaiev = [0.000, 4.0, 7.3, 12.8, 18.9, 19.9, 28.9, 38.7, 64.3]; # ωi values in eV from Werner
#γiev = [0.2, 1.5, 3.3, 11.8, 71.0, 2.9, 3.9, 13.0, 51.9];# γi values in eV from Werner
#fiev = [113.1, 44.6, 54.8, 184.9, 728.1, 65.7, 50.0, 74.7, 544.0];# fi values in eV from Werner

### Bi Werner

#### Bi Werner parameters

In [285]:
#omegaiev = [0.000, 3.9, 6.6, 11.3, 24.1, 27.5, 40.2, 55.8, 84.2]; # ωi values in eV from Werner
#γiev = [0.087, 1.8, 4.5, 6.2, 0.6, 1.9, 9.6, 21.7, 48.4];# γi values in eV from Werner
#fiev = [108.4, 23.6, 50.2, 44.8, 6.6, 13.2, 99.3, 326.8, 373.1];# fi values in eV from Werner

#### Werner dielectric function

In [286]:
#omegai = eV*omegaiev;# ωi values in atomic units
#γi = eV*γiev;# γi values in atomic units
#fi = eV*eV*fiev;# fi values in atomic units

#function epsilon(w) # w is the frequency
#    ωi(w,i) = omegai[i]
#    ϵ1(w) = 1.0 - sum( ((fi[i])*((w*w) - (ωi(w,i)*ωi(w,i))))/( ( ((w*w) - (ωi(w,i)*ωi(w,i)))*((w*w) - (ωi(w,i)*ωi(w,i))) ) + ( w*w*γi[i]*γi[i] ) ) for i in 1:length(omegai))
#    ϵ2(w) = sum( (fi[i]*γi[i]*w)/( ( ((w*w) - (ωi(w,i)*ωi(w,i)))*((w*w) - (ωi(w,i)*ωi(w,i))) ) + ( w*w*γi[i]*γi[i] ) ) for i in 1:length(omegai))
#    return ( ϵ1(w) ) + ii*( ϵ2(w) )
#end;

# Definitions of functions

In [287]:
betalorentz(v) = v/c; # reduced relativistic speed
gammalorentz(v) = 1.0/sqrt(1.0 - ((v/c)^2)); # Lorentz factor
kk(omega) = omega/c; # wave number in vacuum

In [288]:
#using Pkg
#Pkg.add("SpecialFunctions")

using SpecialFunctions

# for:
# beta(x,y)
# besselk(n,z)

The GNU Scientific Library (GSL) is a collection of routines for numerical computing. The routines have been written from scratch in C, and present a modern Applications Programming Interface (API) for C programmers, allowing wrappers to be written for very high level languages. The source code is distributed under the GNU General Public License.

In [289]:
#using Pkg
#Pkg.add("GSL")

using GSL

# for sf_legendre_Plm(l, m, cos(x)) with m > 0
# sf_fact(n) the factorial
# sf_doublefact(n) the double factorial

In [290]:
#using Pkg
#Pkg.add("FastGaussQuadrature")

using FastGaussQuadrature

# Quantities $I^{\ell,m}_{i_1,i_2}$

First we need to define Double factorial $n!!$

(I need to define it to include negative odd arguments). In particular we will need -1!! for $I^{\ell,0}_{0,0}$.

In [291]:
function mydoublefactorial(n)
    if n >= 0
        sf_doublefact(n)
    elseif n == -1
        1.0
    else
       ((-1)^(-0.5*(n+1))) / mydoublefactorial(-n-2) 
    end
end;

I have tabulated all the necessary values of irecur(i1,i2,l,m) and defined a dictionary {key => value} called l100-Ii1i2lm-julia-dictionary.jl

This will allow faster and more economic computations

It is noteworthy that for l<29 the recursive definition is the fastest. After that, for l>=29, it is better to use the integral expression (coupling integrals) defined in 1999 G de Abajo's work. Pay attention to the (-1)^m phase of Condon and Shortley. 

In [292]:
include("l100-Ii1i2lm-julia-dictionary.jl");

# Quantities $\alpha_{\ell,m}$

First I need to define my factorial function since the one in Combinatorics library overflows $\alpha$ for $\ell=11$, $m=11$. I extend the factorial in GSL by defining its non integer values with the gamma function.

In [293]:
function myfactorial(n)
    if typeof(n) == Int64
        sf_fact(n)
    else
        sf_gamma(n+1) 
    end
end;

OLD FACTORIAL

function myfactorial(n)
    if n >=0
        if n < 21
            factorial(n)
        elseif typeof(n) == Int64
            sf_fact(n)
        else
            sf_gamma(n+1)
        end
    else
        sf_gamma(n+1)
    end
end;

In [294]:
function alpha(l,m)
    sqrt(((2*l+1)*myfactorial(l-m))/(4*pi*myfactorial(l+m)))
end;

# Quantities $A^{+}_{\ell,m}$

In [295]:
function AAplus(l,m,v)
    if l < m #para que no tengan broncas las funciones $B_{l,m}$
        0.0
    elseif m >= 0
        (betalorentz(v)^(-l-1))*sum(((ii^(l-j))*mydoublefactorial(2*l+1)*alpha(l,m)*irecur[(j,l-j,l,m)])/((gammalorentz(v)^j)*(2^j)*(myfactorial(l-j))*(myfactorial(0.5*(j-m)))*(myfactorial(0.5*(j+m)))) for j in m:l)
    else
        ((-1)^abs(m))*AAplus(l,abs(m),v)
    end
end;

# Quantities $B_{\ell,m}$

In [296]:
function BB(l,m,v)
    (AAplus(l, m + 1, v)*sqrt((l + m + 1)*(l - m))) - (AAplus(l, m - 1, v)*sqrt((l - m + 1)*(l + m)))
end;

# Functions $\psi^{\text{E, ext}}_{\ell,m}$ and $\psi^{\text{M, ext}}_{\ell,m}$ 

In [297]:
psiEext(l,m,omega,v,b) = ((-2.0)*pi*(ii^(1.0-l))*kk(omega)*BB(l,m,v)*besselk(m,(omega*b)/(v*gammalorentz(v))))/(c*gammalorentz(v)*l*(l + 1));

In [298]:
psiMext(l,m,omega,v,b) = ((-4.0)*pi*(ii^(1.0 - l))*kk(omega)*v*m*AAplus(l, m, v)*besselk(m,(omega*b)/(v*gammalorentz(v))))/(((c)^2)*l*(l + 1));

# (Mie) Scattering coefficients.
# Remember Scattering = i * Mie

It is better to use the well defined Bessel functions os SpecialFUnctions library. All the numerical methods based on recurrences for Ricatti-Bessel functions are useful when one deals with dielectric functions in a small window of frequencies.

I learned that upward recurrence is fast but very inestable. In contrast, downward recurrence is always stable but it is very VERY slow. Therefore I choose to use the special functions as defined by experts.

First we define the Ricatti-Bessel functions $\psi_n(x)$ and $\xi_n(x)$

In [299]:
j(ν, x) = √(π/2x)*besselj(ν+1/2, x); #spherical bessel functions defined from SpecialFunctions besselj and bessely 
y(ν, x) = √(π/2x)*bessely(ν+1/2, x);
h1(ν, x) = j(ν, x) + ii*y(ν, x); #spherical hankel function 1
ψ(ν, x) = x*j(ν, x); # Ricatti-Bessel function \psi
ξ(ν, x) = x*h1(ν, x); # Ricatti-Bessel function \xi

Definition of Messiah's $h_\ell ^{(+)}(x)$

In [300]:
hplus(ν, x) = ii * j(ν, x) - y(ν, x);

Now we define the ratio function $r_n(mx)$ defined on Hong Du paper "Mie-scattering calculation" in Applied Optics from April 2004, DOI: 10.1364/AO.43.001951 

In [301]:
r(n,x) = ψ(n-1, x)/ψ(n, x);

# Mie coefficients $a_n$ and $b_n$ as defined in Hong Du's paper

In [302]:
function miea(n,w,R) # w is omega and R is the radius
    m(w) = sqrt(epsilon(w))
    x(w,R) = (R*w)/c
    factor(n,w,R) = (r(n,m(w)*x(w,R))/m(w)) + (n*(1.0 - (1.0/epsilon(w)))/x(w,R))
    numerator(n,w,R) = (factor(n,w,R) * ψ(n, x(w,R))) - ψ(n-1, x(w,R))
    denominator(n,w,R) = (factor(n,w,R) * ξ(n, x(w,R))) - ξ(n-1, x(w,R))
    return (numerator(n,w,R))/(denominator(n,w,R))
end;

function mieb(n,w,R) # w is omega and R is the radius
    m(w) = sqrt(epsilon(w))
    x(w,R) = (R*w)/c
    numerator(n,w,R) = (r(n,m(w)*x(w,R))*m(w)*ψ(n, x(w,R))) - ψ(n-1, x(w,R))
    denominator(n,w,R) = (r(n,m(w)*x(w,R))*m(w)*ξ(n, x(w,R))) - ξ(n-1, x(w,R))
    return (numerator(n,w,R))/(denominator(n,w,R))
end;

Scattering coefficients $t^E_\ell$ and $t^M_\ell$

In [303]:
tE(l,w,R) = ii * miea(l,w,R);
tM(l,w,R) = ii * mieb(l,w,R);

# Quantities $C^M$ and $D^E$ (Scattered Fields)

In [304]:
CM(l,m,w,v,b,R) = (ii^l)*alpha(l,m)*tM(l,w,R)*psiMext(l,m,w,v,b); # R is NP radius
DE(l,m,w,v,b,R) = (ii^l)*alpha(l,m)*tE(l,w,R)*psiEext(l,m,w,v,b);

# Calculation of scattered electromagnetic fields

Legendre Associated function $P_\ell^m(\cos(\theta))$ from GSL library. I picked the one that agrees with the definition of Wolfram Mathematica. I completed the function to include negative values of $m$.

In [305]:
function P(l,m,x)
    if m >= 0
        sf_legendre_Plm(l, m, cos(x))
    else
        n = -m
        (((-1) ^ n) * P(l,n,x) * myfactorial(l - n)) / myfactorial(l + n)
    end
end;

$\ell$-th contributions

In [306]:
lEscatspherr(l,v,b,R,w,r,θ,ϕ) = sum( (exp(ii*m*ϕ)*DE(l,m,w,v,b,R)*l*(l+1.0)*P(l,m,θ)*hplus(l,kk(w)*r))/(kk(w)*r) for m in -l:l);
lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) = sum( (-1.0)*exp(ii*m*ϕ) * ( (CM(l,m,w,v,b,R)*m*hplus(l,kk(w)*r)*P(l,m,θ)*csc(θ)) + (DE(l,m,w,v,b,R)*( ((l+1.0)*cot(θ)*P(l,m,θ)) - ((l-m+1.0)*P(l+1,m,θ)*csc(θ)) ) * ( (((l+1.0)*hplus(l,kk(w)*r))/(kk(w)*r)) - hplus(l+1,kk(w)*r)) )) for m in -l:l);
lEscatspherphi(l,v,b,R,w,r,θ,ϕ) = sum( ii*exp(ii*m*ϕ) * ( (CM(l,m,w,v,b,R)*hplus(l,kk(w)*r) * ( ((l+1.0)*cot(θ)*P(l,m,θ)) - ((l-m+1.0)*csc(θ)*P(l+1,m,θ))) ) + ( DE(l,m,w,v,b,R)*m*csc(θ)*P(l,m,θ) * ( (((l+1.0)*hplus(l,kk(w)*r))/(kk(w)*r)) - hplus(l+1,kk(w)*r) ) ) )  for m in -l:l);

In [307]:
lHscatspherr(l,v,b,R,w,r,θ,ϕ) = sum( (exp(ii*m*ϕ)*CM(l,m,w,v,b,R)*l*(l+1.0)*P(l,m,θ)*hplus(l,kk(w)*r))/(kk(w)*r) for m in -l:l);
lHscatsphertheta(l,v,b,R,w,r,θ,ϕ) = sum( exp(ii*m*ϕ) * ( (DE(l,m,w,v,b,R)*m*hplus(l,kk(w)*r)*P(l,m,θ)*csc(θ)) - (CM(l,m,w,v,b,R)*( ((l+1.0)*cot(θ)*P(l,m,θ)) - ((l-m+1.0)*P(l+1,m,θ)*csc(θ)) ) * ( (((l+1.0)*hplus(l,kk(w)*r))/(kk(w)*r)) - hplus(l+1,kk(w)*r)) )) for m in -l:l);
lHscatspherphi(l,v,b,R,w,r,θ,ϕ) = sum( ii*exp(ii*m*ϕ) * ( (-DE(l,m,w,v,b,R)*hplus(l,kk(w)*r) * ( ((l+1.0)*cot(θ)*P(l,m,θ)) - ((l-m+1.0)*csc(θ)*P(l+1,m,θ))) ) + ( CM(l,m,w,v,b,R)*m*csc(θ)*P(l,m,θ) * ( (((l+1.0)*hplus(l,kk(w)*r))/(kk(w)*r)) - hplus(l+1,kk(w)*r) ) ) )  for m in -l:l);

# Scattered electromagnetic fields
They compute up to $\ell=20$

In [308]:
Escatspherr(lmax,v,b,R,w,r,θ,ϕ) = sum( lEscatspherr(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);
Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ) = sum( lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);
Escatspherphi(lmax,v,b,R,w,r,θ,ϕ) = sum( lEscatspherphi(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);

In [309]:
Hscatspherr(lmax,v,b,R,w,r,θ,ϕ) = sum( lHscatspherr(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);
Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ) = sum( lHscatsphertheta(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);
Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ) = sum( lHscatspherphi(l,v,b,R,w,r,θ,ϕ) for l in 1:lmax);

In [310]:
LyintegrandEbis(lmax,v,b,R,w,r,θ,ϕ) = sum( (sin(θ)*( (cos(ϕ)*real((lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatsphertheta(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((lEscatspherphi(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextspherphi(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatspherphi(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) ) ) for l in 1:lmax);
LyintegrandEbisref(lmax,v,b,R,w,r,θ,ϕ) = (sin(θ)*( (cos(ϕ)*real((Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((Escatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextspherphi(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) ) );
LyintegrandEref(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*real(ξscatθr(lmax,v,b,R,w,r,θ,ϕ)+ξmixθr(lmax,v,b,R,w,r,θ,ϕ))) - (cos(θ)*sin(ϕ)*real(ξscatϕr(lmax,v,b,R,w,r,θ,ϕ)+ξmixϕr(lmax,v,b,R,w,r,θ,ϕ))) );

# Quantities $A^M$ and $B^E$ (External Fields)

In [311]:
AM(l,m,w,v,b,R) = (ii^l)*alpha(l,m)*psiMext(l,m,w,v,b); # R is NP radius
BE(l,m,w,v,b,R) = (ii^l)*alpha(l,m)*psiEext(l,m,w,v,b);

# External Fields in the Cartesian Exact form

In [312]:
R(x,y,b) = sqrt( (y*y) + ((x-b)*(x-b)));

In [313]:
Eextcartex(v,b,w,x,y,z) = (2*q*abs(w)*(x-b)*exp(ii*w*z/v)*besselk(1,(abs(w)*R(x,y,b))/(v*gammalorentz(v)))) / (R(x,y,b)*v*v*gammalorentz(v));
Eextcartey(v,b,w,x,y,z) = (2*q*abs(w)*(y)*exp(ii*w*z/v)*besselk(1,(abs(w)*R(x,y,b))/(v*gammalorentz(v)))) / (R(x,y,b)*v*v*gammalorentz(v));
Eextcartez(v,b,w,x,y,z) = (-ii*2*q*w*exp(ii*w*z/v)*besselk(0,(abs(w)*R(x,y,b))/(v*gammalorentz(v)))) / (v*v*gammalorentz(v)*gammalorentz(v));

In [314]:
Hextcartex(v,b,w,x,y,z) = (-2*q*betalorentz(v)*abs(w)*(y)*exp(ii*w*z/v)*besselk(1,(abs(w)*R(x,y,b))/(v*gammalorentz(v)))) / (R(x,y,b)*v*v*gammalorentz(v));
Hextcartey(v,b,w,x,y,z) = (2*q*betalorentz(v)*abs(w)*(x-b)*exp(ii*w*z/v)*besselk(1,(abs(w)*R(x,y,b))/(v*gammalorentz(v)))) / (R(x,y,b)*v*v*gammalorentz(v));
Hextcartez(v,b,w,x,y,z) = 0.0;

# External Fields in the Spherical Exact form (from cartesian)

## Spherical to Cartesian coordinates transformation

In [315]:
xxx(r,θ,ϕ) = r*sin(θ)*cos(ϕ);
yyy(r,θ,ϕ) = r*sin(θ)*sin(ϕ);
zzz(r,θ,ϕ) = r*cos(θ);

## External Electric field in spherical exact form (from cartesian)

In [316]:
Eextspherr(v,b,w,r,θ,ϕ) = (Eextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ)*cos(ϕ)) + (Eextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ)*sin(ϕ)) + (Eextcartez(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ));
Eextsphertheta(v,b,w,r,θ,ϕ) = (Eextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ)*cos(ϕ)) + (Eextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ)*sin(ϕ)) - (Eextcartez(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ));
Eextspherphi(v,b,w,r,θ,ϕ) = -(Eextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(ϕ)) + (Eextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(ϕ));

## External Magnetic field in spherical exact form (from cartesian)

In [317]:
Hextspherr(v,b,w,r,θ,ϕ) = (Hextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ)*cos(ϕ)) + (Hextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ)*sin(ϕ)) + (Hextcartez(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ));
Hextsphertheta(v,b,w,r,θ,ϕ) = (Hextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ)*cos(ϕ)) + (Hextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(θ)*sin(ϕ)) - (Hextcartez(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(θ));
Hextspherphi(v,b,w,r,θ,ϕ) = -(Hextcartex(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*sin(ϕ)) + (Hextcartey(v,b,w,xxx(r,θ,ϕ),yyy(r,θ,ϕ),zzz(r,θ,ϕ))*cos(ϕ));

# TENSORS

For the angular momentum transfer only the $\theta r$ and $\phi r$ components are needed. However, I shal aso compute the $rr$ component because it is needed for the LINEAR Momentum transfer.

## Electric tensors

### a) External (which do not contribute)

    ξextθr(v,b,w,r,θ,ϕ) = Eextsphertheta(v,b,w,r,θ,ϕ) * conj(Eextspherr(v,b,w,r,θ,ϕ));
    ξextϕr(v,b,w,r,θ,ϕ) = Eextspherphi(v,b,w,r,θ,ϕ) * conj(Eextspherr(v,b,w,r,θ,ϕ));
    ξextrr(v,b,w,r,θ,ϕ) = 0.5*( (Eextspherr(v,b,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ))) - (Eextsphertheta(v,b,w,r,θ,ϕ)*conj(Eextsphertheta(v,b,w,r,θ,ϕ))) - (Eextspherphi(v,b,w,r,θ,ϕ)*conj(Eextspherphi(v,b,w,r,θ,ϕ))) );

In [318]:
ξextθr(v,b,w,r,θ,ϕ) = 0.0;
ξextϕr(v,b,w,r,θ,ϕ) = 0.0;
ξextrr(v,b,w,r,θ,ϕ) = 0.0;

### b) Scattered

In [319]:
ξscatθr(lmax,v,b,R,w,r,θ,ϕ) = Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ));
ξscatϕr(lmax,v,b,R,w,r,θ,ϕ) = Escatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ));
ξscatrr(lmax,v,b,R,w,r,θ,ϕ) = 0.5*( (Escatspherr(lmax,v,b,R,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) - (Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ))) - (Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Escatspherphi(lmax,v,b,R,w,r,θ,ϕ))) );

### c) Mixed

In [320]:
ξmixθr(lmax,v,b,R,w,r,θ,ϕ) = (Eextsphertheta(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)));
ξmixϕr(lmax,v,b,R,w,r,θ,ϕ) = (Eextspherphi(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)));
ξmixrr(lmax,v,b,R,w,r,θ,ϕ) = 0.5*( ((Eextspherr(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))) - ((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)))+(Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextsphertheta(v,b,w,r,θ,ϕ)))) - ((Eextspherphi(v,b,w,r,θ,ϕ)*conj(Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)))+(Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherphi(v,b,w,r,θ,ϕ)))) );

## Magnetic tensors

### a) External (which do not contribute)

    μextθr(v,b,w,r,θ,ϕ) = Hextsphertheta(v,b,w,r,θ,ϕ) * conj(Hextspherr(v,b,w,r,θ,ϕ));
    μextϕr(v,b,w,r,θ,ϕ) = Hextspherphi(v,b,w,r,θ,ϕ) * conj(Hextspherr(v,b,w,r,θ,ϕ));
    μextrr(v,b,w,r,θ,ϕ) = 0.5*( (Hextspherr(v,b,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ))) - (Hextsphertheta(v,b,w,r,θ,ϕ)*conj(Hextsphertheta(v,b,w,r,θ,ϕ))) - (Hextspherphi(v,b,w,r,θ,ϕ)*conj(Hextspherphi(v,b,w,r,θ,ϕ))) );

In [321]:
μextθr(v,b,w,r,θ,ϕ) = 0.0;
μextϕr(v,b,w,r,θ,ϕ) = 0.0;
μextrr(v,b,w,r,θ,ϕ) = 0.0;

### b) Scattered

In [322]:
μscatθr(lmax,v,b,R,w,r,θ,ϕ) = Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ));
μscatϕr(lmax,v,b,R,w,r,θ,ϕ) = Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ));
μscatrr(lmax,v,b,R,w,r,θ,ϕ) = 0.5*( (Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ))) - (Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ))) - (Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ))) );

### c) Mixed

In [323]:
μmixθr(lmax,v,b,R,w,r,θ,ϕ) = (Hextsphertheta(v,b,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)));
μmixϕr(lmax,v,b,R,w,r,θ,ϕ) = (Hextspherphi(v,b,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)));
μmixrr(lmax,v,b,R,w,r,θ,ϕ) = 0.5*( ((Hextspherr(v,b,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))) - ((Hextsphertheta(v,b,w,r,θ,ϕ)*conj(Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ)))+(Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextsphertheta(v,b,w,r,θ,ϕ)))) - ((Hextspherphi(v,b,w,r,θ,ϕ)*conj(Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ)))+(Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherphi(v,b,w,r,θ,ϕ)))) );

## Electric $D^E$ tensors

In [324]:
DθrE(lmax,v,b,R,w,r,θ,ϕ) = real(ξextθr(v,b,w,r,θ,ϕ)+ξscatθr(lmax,v,b,R,w,r,θ,ϕ)+ξmixθr(lmax,v,b,R,w,r,θ,ϕ));
DϕrE(lmax,v,b,R,w,r,θ,ϕ) = real(ξextϕr(v,b,w,r,θ,ϕ)+ξscatϕr(lmax,v,b,R,w,r,θ,ϕ)+ξmixϕr(lmax,v,b,R,w,r,θ,ϕ));
DrrE(lmax,v,b,R,w,r,θ,ϕ) = real(ξextrr(v,b,w,r,θ,ϕ)+ξscatrr(lmax,v,b,R,w,r,θ,ϕ)+ξmixrr(lmax,v,b,R,w,r,θ,ϕ));

## Magnetic $D^M$ tensors

In [325]:
DθrM(lmax,v,b,R,w,r,θ,ϕ) = real(μextθr(v,b,w,r,θ,ϕ)+μscatθr(lmax,v,b,R,w,r,θ,ϕ)+μmixθr(lmax,v,b,R,w,r,θ,ϕ));
DϕrM(lmax,v,b,R,w,r,θ,ϕ) = real(μextϕr(v,b,w,r,θ,ϕ)+μscatϕr(lmax,v,b,R,w,r,θ,ϕ)+μmixϕr(lmax,v,b,R,w,r,θ,ϕ));
DrrM(lmax,v,b,R,w,r,θ,ϕ) = real(μextrr(v,b,w,r,θ,ϕ)+μscatrr(lmax,v,b,R,w,r,θ,ϕ)+μmixrr(lmax,v,b,R,w,r,θ,ϕ));

## Electromagnetic $D$ tensors (independent) (NOT as a sum of electric and magnetic contributions)

In [326]:
Dθr(lmax,v,b,R,w,r,θ,ϕ) = real(ξextθr(v,b,w,r,θ,ϕ)+ξscatθr(lmax,v,b,R,w,r,θ,ϕ)+ξmixθr(lmax,v,b,R,w,r,θ,ϕ)) + real(μextθr(v,b,w,r,θ,ϕ)+μscatθr(lmax,v,b,R,w,r,θ,ϕ)+μmixθr(lmax,v,b,R,w,r,θ,ϕ));
Dϕr(lmax,v,b,R,w,r,θ,ϕ) = real(ξextϕr(v,b,w,r,θ,ϕ)+ξscatϕr(lmax,v,b,R,w,r,θ,ϕ)+ξmixϕr(lmax,v,b,R,w,r,θ,ϕ)) + real(μextϕr(v,b,w,r,θ,ϕ)+μscatϕr(lmax,v,b,R,w,r,θ,ϕ)+μmixϕr(lmax,v,b,R,w,r,θ,ϕ));
Drr(lmax,v,b,R,w,r,θ,ϕ) = real(ξextrr(v,b,w,r,θ,ϕ)+ξscatrr(lmax,v,b,R,w,r,θ,ϕ)+ξmixrr(lmax,v,b,R,w,r,θ,ϕ)) + real(μextrr(v,b,w,r,θ,ϕ)+μscatrr(lmax,v,b,R,w,r,θ,ϕ)+μmixrr(lmax,v,b,R,w,r,θ,ϕ));

# Integrands for the surface integral [to obtain the spectral contributions  to the ANGULAR Momentum transfer$\mathcal{L}(\omega)$]

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [327]:
Lxintegrand(lmax,v,b,R,w,r,θ,ϕ) = -sin(θ)*( (sin(ϕ)*Dθr(lmax,v,b,R,w,r,θ,ϕ)) + (cos(θ)*cos(ϕ)*Dϕr(lmax,v,b,R,w,r,θ,ϕ)) );
Lyintegrand(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*Dθr(lmax,v,b,R,w,r,θ,ϕ)) - (cos(θ)*sin(ϕ)*Dϕr(lmax,v,b,R,w,r,θ,ϕ)) );
Lzintegrand(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*sin(θ)*Dϕr(lmax,v,b,R,w,r,θ,ϕ);

### ELECTRIC

In [328]:
LxintegrandE(lmax,v,b,R,w,r,θ,ϕ) = -sin(θ)*( (sin(ϕ)*DθrE(lmax,v,b,R,w,r,θ,ϕ)) + (cos(θ)*cos(ϕ)*DϕrE(lmax,v,b,R,w,r,θ,ϕ)) );
LyintegrandE(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*DθrE(lmax,v,b,R,w,r,θ,ϕ)) - (cos(θ)*sin(ϕ)*DϕrE(lmax,v,b,R,w,r,θ,ϕ)) );
LzintegrandE(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*sin(θ)*DϕrE(lmax,v,b,R,w,r,θ,ϕ);

## electric for convergence test

In [329]:
LyintegrandEtest(lmax,l,v,b,R,w,r,θ,ϕ) = (sin(θ)*( (cos(ϕ)*real((lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatsphertheta(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((lEscatspherphi(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextspherphi(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatspherphi(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) ) );


LyintegrandEbis(lmax,v,b,R,w,r,θ,ϕ) = sum( (sin(θ)*( (cos(ϕ)*real((lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatsphertheta(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((lEscatspherphi(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextspherphi(v,b,w,r,θ,ϕ)*conj(lEscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lEscatspherphi(l,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) ) ) for l in 1:lmax);
#LyintegrandEbisref(lmax,v,b,R,w,r,θ,ϕ) = (sin(θ)*( (cos(ϕ)*real((Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextsphertheta(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((Escatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Eextspherphi(v,b,w,r,θ,ϕ)*conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Escatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Eextspherr(v,b,w,r,θ,ϕ)))))) ) );
#LyintegrandEref(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*real(ξscatθr(lmax,v,b,R,w,r,θ,ϕ)+ξmixθr(lmax,v,b,R,w,r,θ,ϕ))) - (cos(θ)*sin(ϕ)*real(ξscatϕr(lmax,v,b,R,w,r,θ,ϕ)+ξmixϕr(lmax,v,b,R,w,r,θ,ϕ))) );

#LyintegrandEref(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*real((lEscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+ξmixθr(lmax,v,b,R,w,r,θ,ϕ))) - (cos(θ)*sin(ϕ)*real((lEscatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Escatspherr(lmax,v,b,R,w,r,θ,ϕ)))+ξmixϕr(lmax,v,b,R,w,r,θ,ϕ))) );

### MAGNETIC

In [331]:
LxintegrandM(lmax,v,b,R,w,r,θ,ϕ) = -sin(θ)*( (sin(ϕ)*DθrM(lmax,v,b,R,w,r,θ,ϕ)) + (cos(θ)*cos(ϕ)*DϕrM(lmax,v,b,R,w,r,θ,ϕ)) );
LyintegrandM(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*DθrM(lmax,v,b,R,w,r,θ,ϕ)) - (cos(θ)*sin(ϕ)*DϕrM(lmax,v,b,R,w,r,θ,ϕ)) );
LzintegrandM(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*sin(θ)*DϕrM(lmax,v,b,R,w,r,θ,ϕ);

## magnetic for convergence test

In [332]:
LyintegrandMtest(lmax,l,v,b,R,w,r,θ,ϕ) =  (sin(θ)*( (cos(ϕ)*real((lHscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextsphertheta(v,b,w,r,θ,ϕ)*conj(lHscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lHscatsphertheta(l,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((lHscatspherphi(l,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextspherphi(v,b,w,r,θ,ϕ)*conj(lHscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lHscatspherphi(l,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) ) );


LyintegrandMbis(lmax,v,b,R,w,r,θ,ϕ) = sum( (sin(θ)*( (cos(ϕ)*real((lHscatsphertheta(l,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextsphertheta(v,b,w,r,θ,ϕ)*conj(lHscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lHscatsphertheta(l,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((lHscatspherphi(l,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextspherphi(v,b,w,r,θ,ϕ)*conj(lHscatspherr(l,v,b,R,w,r,θ,ϕ))) + (lHscatspherphi(l,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) ) ) for l in 1:lmax);
#LyintegrandMbisref(lmax,v,b,R,w,r,θ,ϕ) = (sin(θ)*( (cos(ϕ)*real((Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextsphertheta(v,b,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Hscatsphertheta(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) - (cos(θ)*sin(ϕ)*real((Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ) * conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ)))+((Hextspherphi(v,b,w,r,θ,ϕ)*conj(Hscatspherr(lmax,v,b,R,w,r,θ,ϕ))) + (Hscatspherphi(lmax,v,b,R,w,r,θ,ϕ)*conj(Hextspherr(v,b,w,r,θ,ϕ)))))) ) );
#LyintegrandMref(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*real(μscatθr(lmax,v,b,R,w,r,θ,ϕ)+μmixθr(lmax,v,b,R,w,r,θ,ϕ))) - (cos(θ)*sin(ϕ)*real(μscatϕr(lmax,v,b,R,w,r,θ,ϕ)+μmixϕr(lmax,v,b,R,w,r,θ,ϕ))) );

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [333]:
Pxintegrand(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*(sin(θ)*Drr(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*Dθr(lmax,v,b,R,w,r,θ,ϕ))) - (sin(ϕ)*Dϕr(lmax,v,b,R,w,r,θ,ϕ) )); 
Pyintegrand(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (sin(ϕ)*(sin(θ)*Drr(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*Dθr(lmax,v,b,R,w,r,θ,ϕ))) + (cos(ϕ)*Dϕr(lmax,v,b,R,w,r,θ,ϕ) )); 
Pzintegrand(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*(cos(θ)*Drr(lmax,v,b,R,w,r,θ,ϕ) - sin(θ)*Dθr(lmax,v,b,R,w,r,θ,ϕ)); 

### ELECTRIC

In [334]:
PxintegrandE(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*(sin(θ)*DrrE(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*DθrE(lmax,v,b,R,w,r,θ,ϕ))) - (sin(ϕ)*DϕrE(lmax,v,b,R,w,r,θ,ϕ) )); 
PyintegrandE(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (sin(ϕ)*(sin(θ)*DrrE(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*DθrE(lmax,v,b,R,w,r,θ,ϕ))) + (cos(ϕ)*DϕrE(lmax,v,b,R,w,r,θ,ϕ) )); 
PzintegrandE(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*(cos(θ)*DrrE(lmax,v,b,R,w,r,θ,ϕ) - sin(θ)*DθrE(lmax,v,b,R,w,r,θ,ϕ)); 

### MAGNETIC

In [335]:
PxintegrandM(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (cos(ϕ)*(sin(θ)*DrrM(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*DθrM(lmax,v,b,R,w,r,θ,ϕ))) - (sin(ϕ)*DϕrM(lmax,v,b,R,w,r,θ,ϕ) )); 
PyintegrandM(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*( (sin(ϕ)*(sin(θ)*DrrM(lmax,v,b,R,w,r,θ,ϕ) + cos(θ)*DθrM(lmax,v,b,R,w,r,θ,ϕ))) + (cos(ϕ)*DϕrM(lmax,v,b,R,w,r,θ,ϕ) )); 
PzintegrandM(lmax,v,b,R,w,r,θ,ϕ) = sin(θ)*(cos(θ)*DrrM(lmax,v,b,R,w,r,θ,ϕ) - sin(θ)*DθrM(lmax,v,b,R,w,r,θ,ϕ)); 

# Spectral contributions $\mathcal{L}(\omega)$ and $\mathcal{P}(\omega)$ (surface integrals)

# I found possible and better to compute multidimensional integrals using cubatures.

### Cuba is a suite to compute cubatures in the hypercube $[0,1]^n$. See :

https://giordano.github.io/Cuba.jl/stable/#Usage-1

https://github.com/giordano/Cuba.jl

http://www.feynarts.de/cuba/



In particular, in https://giordano.github.io/Cuba.jl/stable/#Usage-1 I found that it is possible to do all the integrals as:

$$\int_a^b f(x)dx = \int_0^1 f(a+[b-a]y)(b-a)dy$$
$$\int_a^\infty f(x)dx = \int_0^1 f\left(a+\frac{y}{1-y}\right)(b-a)\frac{1}{(1-y)^2}dy,$$

so that 

$$\int_0^\infty \int_0^\pi \int_0^{2\pi}   f(\theta,\phi,\omega) d\omega d\phi d\theta
= 
2\pi^2
\int_0^1\int_0^1\int_0^1 f\left( \pi y_\theta, 2\pi y_\phi,\frac{y_\omega}{1-y_\omega}\right)\left(\frac{1}{1-y_\omega}\right)^2 dy_\omega dy_\phi dy_\theta$$

In [336]:
#using Pkg
#Pkg.add("Cuba")
using Cuba;

### rtol (type: Real, default: 1e-4), and atol (type: Real, default: 1e-12): the requested relative ($\varepsilon_{\text{rel}}$) and absolute ($\varepsilon_{\text{abs}}$) accuracies. The integrator tries to find an estimate $\hat{I}$ for the integral $I$ which for every component $c$ fulfills $∣\hat{I}_c−I_c∣\leq \max(\varepsilon_{\text{abs}}, \varepsilon_{\text{rel}}∣I_c∣)$.

cuhre(integrand, ndim=2, ncomp=1[, keywords]) -> integral, error, probability, neval, fail, nregions

Arguments

The only mandatory argument of integrator functions is:

    integrand (type: Function): the function to be integrated

Optional positional arguments are:

    ndim (type: Integer): the number of dimensions of the integratation domain. If omitted, defaults to 1 in vegas and suave, to 2 in divonne and cuhre. Note: ndim must be at least 2 with the latest two methods.
    
    ncomp (type: Integer): the number of components of the integrand. Default to 1 if omitted

integrand should be a function integrand(x, f) taking two arguments:

    the input vector x of length ndim
    the output vector f of length ncomp, used to set the value of each component of the integrand at point x

x and f are matrices with dimensions (ndim, nvec) and (ncomp, nvec), respectively, when nvec > 1. See the Vectorization section below for more information.

### rtol (type: Real, default: 1e-4), and atol (type: Real, default: 1e-12): the requested relative ($\varepsilon_{\text{rel}}$) and absolute ($\varepsilon_{\text{abs}}$) accuracies. The integrator tries to find an estimate $\hat{I}$ for the integral $I$ which for every component $c$ fulfills $∣\hat{I}_c−I_c∣\leq \max(\varepsilon_{\text{abs}}, \varepsilon_{\text{rel}}∣I_c∣)$.

minevals (type: Real, default: 0): the minimum number of integrand evaluations required

maxevals (type: Real, default: 1000000): the (approximate) maximum number of integrand evaluations allowed

# Definition of the spectral contributions $\mathcal{L}_x(\omega)$, $\mathcal{L}_x(\omega)$ and $\mathcal{L}_x(\omega)$

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [337]:
maximumevaluations = 40000000;

In [338]:
Lxspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*Lxintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Lyspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*Lyintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Lzspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*Lzintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### ELECTRIC

In [339]:
LxspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LxintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
LyspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LyintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
LzspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LzintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### MAGNETIC

In [340]:
LxspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LxintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
LyspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LyintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
LzspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*r*LzintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

# Definition of the spectral contributions $\mathcal{P}_x(\omega)$, $\mathcal{P}_x(\omega)$ and $\mathcal{P}_x(\omega)$

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [341]:
Pxspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*Pxintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Pyspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*Pyintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Pzspectral(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*Pzintegrand(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### ELECTRIC

In [342]:
PxspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PxintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
PyspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PyintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
PzspectralE(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PzintegrandE(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### MAGNETIC

In [343]:
PxspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PxintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
PyspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PyintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
PzspectralM(lmax,v,b,R,w,r) = cuhre((x, f) -> f[1] = 0.5*r*r*PzintegrandM(lmax,v,b,R,w,r,pi*x[1],2.0*pi*x[2]),2,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

# Definition of the total ANGULAR momentum transfer $\Delta\vec{\mathbf{L}}=L_x\hat{\mathbf{e}}_x +L_y\hat{\mathbf{e}}_y +L_z\hat{\mathbf{e}}_z$

OBS: $$L_x = 0$$
$$L_y = \Delta L_\text{tot}$$ 
$$L_z=0.$$

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [344]:
Lxcomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Lxintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Lycomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Lyintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Lzcomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Lzintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### ELECTRIC

In [345]:
LxcompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LxintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
LycompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
LzcompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LzintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

## electric for convergence test

In [346]:
LycompleteEbis(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandEbis(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

LycompleteEtest(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandEtest(lmax,lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);


### MAGNETIC

In [347]:
LxcompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LxintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
LycompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
LzcompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LzintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

## magnetic for convergence test

In [348]:
LycompleteMbis(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandMbis(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

LycompleteMtest(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*LyintegrandMtest(lmax,lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);


# Definition of the total LINEAR momentum transfer $\Delta\vec{\mathbf{P}}=P_x\hat{\mathbf{e}}_x +P_y\hat{\mathbf{e}}_y +P_z\hat{\mathbf{e}}_z$
OBS: $$P_x = \Delta P_{\perp}$$
$$P_y = 0$$ 
$$P_z=\Delta P_{\parallel}.$$

### TOTALS (independent) (NOT as a sum of electric and magnetic contributions)

In [349]:
Pxcomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Pxintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Pycomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Pyintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);
Pzcomplete(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*Pzintegrand(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-12, rtol=1e-3,maxevals = maximumevaluations);

### ELECTRIC

In [350]:
PxcompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PxintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
PycompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PyintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
PzcompleteE(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PzintegrandE(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

### MAGNETIC

In [351]:
PxcompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PxintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
PycompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PyintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);
PzcompleteM(lmax,v,b,NPRadius,r) = cuhre((x, f) -> f[1] = ((0.5*r*r*1.0)/((1.0-x[3])*(1.0-x[3])))*PzintegrandM(lmax,v,b,NPRadius,(x[3])/(1.0-x[3]),r,pi*x[1],2.0*pi*x[2]),3,1,atol=1e-14, rtol=1e-4,maxevals = maximumevaluations);

    check http://www.pkofod.com/2017/04/24/timing-in-julia/
    
    gctime = time julia spends freeing memory
    
    elarray2 = Lycomplete(1,0.5*c,20*nm,nm,1.1*nm)

    Component:
     1: -1.6196156990488162e-6 ± 1.4959353170820283e-9 (prob.: 0.0018956097984944632)
    Integrand evaluations: 13589
    Number of subregions:  54
    Note: The desired accuracy was reached

    elarray2[1][1]
    -1.6196156990488162e-6

    elarray2[2][1]
    1.4959353170820283e-9

    elarray2[3][1]
    0.0018956097984944632

    elarray2[4][1]
    13589

    elarray2[5][1] (fail type)
    0

    elarray2[6][1]
    54
    
    
        Lyarray, t, bytes, gctime, memallocs = @timed Lycomplete(1,0.5*c,20*nm,nm,1.1*nm)

    (Component:
     1: -1.6196156990488162e-6 ± 1.4959353170820283e-9 (prob.: 0.0018956097984944632)
    Integrand evaluations: 13589
    Number of subregions:  54
    Note: The desired accuracy was reached, 11.945871031, 5065762368, 1.241923489, Base.GC_Diff(5065762368, 0, 0, 212708629, 0, 0, 1241923489, 221, 0))

    Lyarray
    Component:
     1: -1.6196156990488162e-6 ± 1.4959353170820283e-9 (prob.: 0.0018956097984944632)
    Integrand evaluations: 13589
    Number of subregions:  54
    Note: The desired accuracy was reached

    t
    11.945871031

    bytes
    5065762368

    gctime
    1.241923489

    memallocs
    Base.GC_Diff(5065762368, 0, 0, 212708629, 0, 0, 1241923489, 221, 0)

    fail (type: Int32): an error flag:

        fail = 0, the desired accuracy was reached
        fail = -1, dimension out of range
        fail > 0, the accuracy goal was not met within the allowed maximum number of integrand evaluations. While Vegas, Suave, and Cuhre simply return 1, Divonne can estimate the number of points by which maxevals needs to be increased to reach the desired accuracy and returns this value.

### To write on a file:

In [352]:
using Printf # see https://www.geeksforgeeks.org/formatting-of-strings-in-julia/ for formats

    %c: a single character (letter, number, special symbol)
    %s: string (a combination of characters)
    %d: integer (any whole number (not a fraction))
    %f: float (numbers having floating decimal points)
    %.nf: float restricted up to n decimal places
    %e: scientific representation of a float number

## To write on a file 
### from https://docs.julialang.org/en/v1/base/io-network/

    open(filename::AbstractString, [mode::AbstractString]; lock = true) -> IOStream
    Alternate syntax for open, where a string-based mode specifier is used instead of the five booleans. The values of mode correspond to those from fopen(3) or Perl open, and are equivalent to setting the following boolean groups:

    Mode	Description	Keywords
    r	read	none
    w	write, create, truncate	write = true
    a	write, create, append	append = true
    r+	read, write	read = true, write = true
    w+	read, write, create, truncate	truncate = true, read = true
    a+	read, write, create, append	append = true, read = true
    The lock keyword argument controls whether operations will be locked for safe multi-threaded access.

# Angular momentum transfer l-multipole contribution at a fixed speed convergence

    outfile = "Al-a1nm-v01c-b1p5nm-l-from-1-to-11-convergence.dat"
    f = open(outfile, "a")
    @timed 1+1;
    @printf(f,"%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s\n","lmax", "v/c", "b[nm]", "NPradius[nm]", "integration-radius", "LE[hbar]","LEerror", "LEerrorprob","LEtime[s]","LEevals","LEfail","LEsubregions","LEbytes","LEgctime[s]", "LM[hbar]","LMerror", "LMerrorprob","LMtime[s]","LMevals","LMfail","LMsubregions","LMbytes","LMgctime[s]","LTOTAL[hbar]")
    close(f)
    @printf("%s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s %s\n","lmax", "v/c", "b[nm]", "NPradius[nm]", "integration-radius", "LE[hbar]","LEerror", "LEerrorprob","LEtime[s]","LEevals","LEfail","LEsubregions","LEbytes","LEgctime[s]", "LM[hbar]","LMerror", "LMerrorprob","LMtime[s]","LMevals","LMfail","LMsubregions","LMbytes","LMgctime[s]","LTOTAL[hbar]")

    ellmax = 11;
    electronspeed = 0.1;
    impactparameter = 1.5;
    nanoparticleradius = 1.0;
    radiusofintegration = 1.1;

    ELyarray, Et, Ebytes, Egctime, Ememallocs = @timed LycompleteEtest(1,electronspeed*c,impactparameter*nm,nanoparticleradius*nm,radiusofintegration*nm);
    MLyarray, Mt, Mbytes, Mgctime, Mmemallocs = @timed LycompleteMtest(1,electronspeed*c,impactparameter*nm,nanoparticleradius*nm,radiusofintegration*nm);

    TMAelectric = ELyarray[1][1];
    TMAmagnetic = MLyarray[1][1];
    TMAtotal = TMAelectric + TMAmagnetic;

    f = open(outfile, "a")
    @printf(f,"%d %.3f %.2f %.2f %.2f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e\n", 1, electronspeed, impactparameter,nanoparticleradius,radiusofintegration,TMAelectric,ELyarray[2][1],ELyarray[3][1],Et,ELyarray[4][1],ELyarray[5][1],ELyarray[6][1],Ebytes,Egctime,TMAmagnetic,MLyarray[2][1],MLyarray[3][1],Mt,MLyarray[4][1],MLyarray[5][1],MLyarray[6][1],Mbytes,Mgctime,TMAtotal)  
    close(f)

    @printf("%f %s\n", 1,":") 
    @printf("%d %.3f %.2f %.2f %.2f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e\n", 1, electronspeed, impactparameter,nanoparticleradius,radiusofintegration,TMAelectric,ELyarray[2][1],ELyarray[3][1],Et,ELyarray[4][1],ELyarray[5][1],ELyarray[6][1],Ebytes,Egctime,TMAmagnetic,MLyarray[2][1],MLyarray[3][1],Mt,MLyarray[4][1],MLyarray[5][1],MLyarray[6][1],Mbytes,Mgctime,TMAtotal)

    for j in 2:ellmax
        ELyarray, Et, Ebytes, Egctime, Ememallocs = @timed LycompleteEtest(j,electronspeed*c,impactparameter*nm,nanoparticleradius*nm,radiusofintegration*nm);
        MLyarray, Mt, Mbytes, Mgctime, Mmemallocs = @timed LycompleteMtest(j,electronspeed*c,impactparameter*nm,nanoparticleradius*nm,radiusofintegration*nm);
        TMAelectric = TMAelectric + ELyarray[1][1];
        TMAmagnetic = TMAmagnetic + MLyarray[1][1];
        TMAtotal = TMAtotal +  ELyarray[1][1] + MLyarray[1][1];
        f = open(outfile, "a")
        @printf(f,"%d %.3f %.2f %.2f %.2f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e\n", j, electronspeed, impactparameter,nanoparticleradius,radiusofintegration,TMAelectric,ELyarray[2][1],ELyarray[3][1],Et,ELyarray[4][1],ELyarray[5][1],ELyarray[6][1],Ebytes,Egctime,TMAmagnetic,MLyarray[2][1],MLyarray[3][1],Mt,MLyarray[4][1],MLyarray[5][1],MLyarray[6][1],Mbytes,Mgctime,TMAtotal)
        close(f)
        @printf("%f %s\n", j,":")
        @printf("%d %.3f %.2f %.2f %.2f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e %.3e %.3e %.3f %d %d %d %d %.3f %.5e\n", j, electronspeed, impactparameter,nanoparticleradius,radiusofintegration,TMAelectric,ELyarray[2][1],ELyarray[3][1],Et,ELyarray[4][1],ELyarray[5][1],ELyarray[6][1],Ebytes,Egctime,TMAmagnetic,MLyarray[2][1],MLyarray[3][1],Mt,MLyarray[4][1],MLyarray[5][1],MLyarray[6][1],Mbytes,Mgctime,TMAtotal)
    end